In [1]:
from pathlib import Path
import pandas as pd
import json


In [3]:
# All of this data is just for Canada excluding territories

datadir = Path("./raw_data/distribution")
csv_files = list(datadir.glob("*/*.csv"))
nutrient_directories = list(datadir.glob("*"))
reference_file = datadir / "DistributionReferenceValues-EN.csv"

In [4]:
len(csv_files)

896

In [9]:
print(sorted([x.name for x in nutrient_directories if x.is_dir()]))

['10_Folate', '11_Vitamin_B12', '12_Total_carbohydrates', '13_Total_sugars', '14_VitaminC', '15_Zinc', '16_Magnesium', '17_VitaminA', '18_Percent_total_energy_from_carbohydrates', '19_Percent_total_energy_from_fats', '1_Calcium', '20_Percent_total_energy_from_protein', '21_Cholesterol', '22_Phosphorus', '23_VitaminB6', '24_Niacin', '25_Riboflavin', '26_Thiamin', '27_Percent_total_energy_from_linoleic_fatty', '28_Percent_total_energy_from_linolenic_fatty', '2_VitaminD', '3_Iron', '4_Sodium', '5_Potassium', '6_Percent_total_energy_from_saturated_fat', '7_Percent_total_energy_from_sugars', '8_Total_Energy', '9_Fibre']


In [17]:
nutrient_dict = {
    '1_Calcium': 'Calcium (mg/d)',
    '2_VitaminD': 'Vitamin D (mcg/d)',
    '3_Iron': 'Iron (mg/d)',
    '4_Sodium': 'Sodium (mg/d)',
    '5_Potassium': 'Potassium (mg/d)',
    '6_Percent_total_energy_from_saturated_fat': 'Percentage of total energy intake from saturated fats',
    '7_Percent_total_energy_from_sugars': 'Percentage of total energy intake from sugars',
    '8_Total_Energy': 'Total energy intake (kcal/d)',
    '9_Fibre': 'Total dietary fibre (g/d)',
    '10_Folate': 'Folate (DFE/d)',
    '11_Vitamin_B12': 'Vitamin B12 (mcg/d)',
    '12_Total_carbohydrates': 'Total carbohydrates (g/d)',
    '13_Total_sugars': 'Total sugars (g/d)',
    '14_VitaminC': 'Vitamin C (mg/d)',
    '15_Zinc': 'Zinc (mg/d)',
    '16_Magnesium': 'Magnesium (mg/d)',
    '17_VitaminA': 'Vitamin A (RAE/d)',
    '18_Percent_total_energy_from_carbohydrates': 'Percentage of total energy intake from carbohydrates',
    '19_Percent_total_energy_from_fats': 'Percentage of total energy intake from fat',
    '20_Percent_total_energy_from_protein': 'Percentage of total energy intake from protein',
    '21_Cholesterol': 'Cholesterol (mg/d)',
    '22_Phosphorus': 'Phosphorus (mg/d)',
    '23_VitaminB6': 'Vitamin B6 (mg/d)',
    '24_Niacin': 'Niacin (NE/d)',
    '25_Riboflavin': 'Riboflavin (mg/d)',
    '26_Thiamin': 'Thiamin (mg/d)',
    '27_Percent_total_energy_from_linoleic_fatty': 'Percentage of total energy intake from linoleic acid',
    '28_Percent_total_energy_from_linolenic_fatty': 'Percentage of total energy intake from linolenic acid'
}
print(len(nutrient_dict))

28


In [22]:
def parse_nutrient_filename(f: Path):
    fstripped = f.with_suffix("").name
    if "Male" in fstripped:
        sex = "Male"
        fstripped.replace("Male","")
    elif "Female" in fstripped:
        sex = "Female"
    else:
        sex = "Both"
    fstripped = fstripped.replace("Male","").replace("Female","")
    
    if "19_more" in fstripped:
        fstripped = fstripped.replace("19_more", "19 and over")
    elif "71_more" in fstripped:
        fstripped = fstripped.replace("71_more", "71 and over")
    
    components = fstripped.split("_")
    nutrient = nutrient_dict[f.parent.name]
    code = components[0]
    year = components[1].replace("cchs","")
    group = components[2].replace("To","-").replace("years","")
    
    return nutrient, code, year, group, sex

def parse_csv_to_df(csv: Path, nutrient: str, code: str, year: str, group: str, sex: str, subsample_n: int, ref_code: int):
    df = pd.read_csv(csv)
    df['Nutrient/Item (unit)'] = nutrient
    df['Year'] = year
    df['Reg_Prov'] = "Canada excluding territories"
    df['Sex'] = sex
    df['Age (years)'] = group
    df['code'] = code
    df['ref_code'] = ref_code
    if subsample_n is not None:
        df = subsample_df(df, subsample_n)
    return df

def subsample_df(df: pd.DataFrame, n: int):
    # n is how many rows we want to skip
    df = df.iloc[::n,:]
    return df

In [23]:
"""
Coding for each unique group (e.g. combination of Nutrient/Sex/Age) is stored in the Ref-code column of ./distribution/DistributionReferenceValues-EN.csv

Contents of the file are dumped to JSON in the following commands.
"""
reference_file = datadir / "DistributionReferenceValues-EN.csv"
ref_df = pd.read_csv(reference_file)
ref_df_tiny = ref_df[['Ref-code', 'Adequacy-Value', 'Adequacy-Type', 'Excess-Value', 'Excess-Type']]
val_col_name = 'Ref-code'
out_json = datadir / "DistributionReferenceValues-EN.json"
with open(str(out_json), 'w') as f:
    json.dump([{'Ref-code': i, 'metadata': {j: row[j] for j in ref_df_tiny.columns if j != val_col_name}, val_col_name: row[val_col_name]} for i, row in ref_df_tiny.iterrows()], f, indent=4)

In [25]:
df_list = []
subsample_n = 20  # e.g. take every 10th row (none or int > 0)
for f in csv_files:
    nutrient, code, year, group, sex = parse_nutrient_filename(f)
    ref_code = ref_df[ref_df['Nutrient/Item (unit)'] == nutrient][ref_df['Sex'] == sex][ref_df['Age (years)'] == group]['Ref-code'].values[0]
    df = parse_csv_to_df(f, nutrient, code, year, group, sex, subsample_n, ref_code)
    df_list.append(df)
master_df = pd.concat(df_list)

/home/forest/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [33]:
master_df['Age (years)'] = df['Age (years)'].replace(['-'], ' to ', regex=True)
master_df.to_csv(str(datadir / "distributions-en-20th.csv"), index=None)
master_df.head()

,x,y,Nutrient/Item (unit),Year,Reg_Prov,Sex,Age (years),code,ref_code
0,37.088759,0.000002,Magnesium (mg/d),2004,Canada excluding territories,Male,19 to 30,8,246
20,68.862449,0.000014,Magnesium (mg/d),2004,Canada excluding territories,Male,19 to 30,8,246
40,100.636139,0.000073,Magnesium (mg/d),2004,Canada excluding territories,Male,19 to 30,8,246
60,132.409829,0.000255,Magnesium (mg/d),2004,Canada excluding territories,Male,19 to 30,8,246
80,164.183519,0.000627,Magnesium (mg/d),2004,Canada excluding territories,Male,19 to 30,8,246
